In [1]:
# visualizar a versão instalada
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
#detalhes do ambiente
# não esquecer de modificar o ambiente para GPU no colab
!nvidia-smi

Thu Jan  5 19:48:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#instalando um plugin para escrever os codigos cuda no jupyter
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-w8dicvww
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-w8dicvww
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=8e549cd6e2f3a5762604319ba8183cb6d981f7fb1093a526dab07e04c1a4790b
  Stored in directory: /tmp/pip-ephem-wheel-cache-sgx3blqa/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [4]:
# o prefixo abaixo faz com que as celulas sejam executadas pelo compilador do Cuda nvcc
%%cu
#include <iostream>
int main() {
  int nDevices;
	std::cout << "Welcome to CUDA!" << std::endl;
  cudaGetDeviceCount(&nDevices);
  for (int i = 0; i < nDevices; i++) {
    //cria um objeto de propriedades, chama uma função para popular e depois exibe
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, i);
    std::cout << "Device Number: " << i << std::endl;
    std::cout << "  Device name: " << prop.name << std::endl;
    std::cout << "  Memory Clock Rate (KHz): " << prop.memoryClockRate << std::endl;
    std::cout << "  Memory Bus Width (bits): " << prop.memoryBusWidth << std::endl;
    std::cout << "  Peak Memory Bandwidth (GB/s): " << 2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6 << std::endl;
  }
	return 0;
}

Welcome to CUDA!
Device Number: 0
  Device name: Tesla T4
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064



In [5]:
# a tag abaixo gera um arquivo para execução em cuda
%%writefile 01-hello-gpu.cu
#include <stdio.h>

void CPUFunction() {
  printf("Esta função está definida para ser executada na CPU.\n");
}
//a tag global define o escopo da função 
// funções cuda devem retornar tipo void
__global__ void GPUFunction() {
  printf("Esta função está definida para ser executada na GPU.\n");
}

int main() {
  CPUFunction();
  //para chamar uma função para GPU (kernel) é necessário passar o numero de blocos e de threads dentro de <<>>
  GPUFunction<<<1, 1>>>();
  // então sincroniza todas as threads antes de encerrar o programa
  // execuções da CPU também ficam esperando para executarem
  cudaDeviceSynchronize();
  return 0;
}

Writing 01-hello-gpu.cu


In [6]:
!nvcc -arch=sm_70 -o hello-gpu 01-hello-gpu.cu -run

Esta função está definida para ser executada na CPU.
Esta função está definida para ser executada na GPU.


In [7]:
# exercício para aprender sobre id de threads e numero de blocos
# a quantidade de threads marcada será configurada para cada bloco adicionado
%%writefile 02-thread-and-block-idx.cu
#include <stdio.h>

__global__ void printSuccessForCorrectExecutionConfiguration() {
  //a mensagem só será acionada quando chegar na thread identificada, no ultimo bloco
  //lembrando que começa do zero
  if(threadIdx.x == 1023 && blockIdx.x == 255)
    printf("Success!\n");
}
//mas se eu adicionar da mesma forma sem confirmar o bloco terei
__global__ void printSuccessForPartialCorrectExecutionConfiguration() {
  //a mensagem só será acionada quando chegar na thread identificada, no ultimo bloco
  //lembrando que começa do zero
  if(threadIdx.x == 1023)
    printf("Success for  %i !\n", blockIdx.x);
}

int main() {
  //este número vai fazer a configuração certa para ativar a mensagem da função
  printSuccessForCorrectExecutionConfiguration<<<256, 1024>>>();
  cudaDeviceSynchronize();
  printSuccessForPartialCorrectExecutionConfiguration<<<256, 1024>>>();
  cudaDeviceSynchronize();
  return 0;
}


Writing 02-thread-and-block-idx.cu


In [8]:
!nvcc -arch=sm_70 -o thread-and-block-idx 02-thread-and-block-idx.cu -run

Success!
Success for  19 !
Success for  39 !
Success for  14 !
Success for  34 !
Success for  16 !
Success for  36 !
Success for  29 !
Success for  9 !
Success for  11 !
Success for  31 !
Success for  4 !
Success for  24 !
Success for  6 !
Success for  26 !
Success for  1 !
Success for  21 !
Success for  17 !
Success for  37 !
Success for  12 !
Success for  32 !
Success for  7 !
Success for  27 !
Success for  2 !
Success for  22 !
Success for  15 !
Success for  35 !
Success for  10 !
Success for  30 !
Success for  5 !
Success for  25 !
Success for  0 !
Success for  20 !
Success for  18 !
Success for  38 !
Success for  13 !
Success for  33 !
Success for  8 !
Success for  28 !
Success for  3 !
Success for  23 !
Success for  40 !
Success for  41 !
Success for  42 !
Success for  43 !
Success for  44 !
Success for  45 !
Success for  46 !
Success for  50 !
Success for  48 !
Success for  49 !
Success for  47 !
Success for  53 !
Success for  54 !
Success for  51 !
Success for  55 !
Success for

In [9]:
# paralelização de loops
# a função deve ser feita para uma thread executar e então o numero de loops é dado pelo número de threads definidas
%%writefile 03-single-block-loop.cu
#include <stdio.h>

__global__ void loop() {
  /* This kernel does the work of only 1 iteration
   * of the original for loop. Indication of which
   * "iteration" is being executed by this kernel is
   * still available via `threadIdx.x`.  */
  printf("This is iteration number %d\n", threadIdx.x);
}

int main() {
  /* It is the execution context that sets how many "iterations"
   * of the "loop" will be done.
   */
  loop<<<1, 10>>>();
  cudaDeviceSynchronize();
}

Writing 03-single-block-loop.cu


In [10]:
!nvcc -arch=sm_70 -o single-block-loop 03-single-block-loop.cu -run

This is iteration number 0
This is iteration number 1
This is iteration number 2
This is iteration number 3
This is iteration number 4
This is iteration number 5
This is iteration number 6
This is iteration number 7
This is iteration number 8
This is iteration number 9


In [11]:
# existe um limite de 1024 threads por blocos
# então podemos usar as threads dos outros blocos para simular um bloco de mais threads
# o truque é feito através de "blockIdx.x * blockDim.x + threadIdx.x"
# gerando um id sequencial para todas as threads, mesmo de blocos diferentes
%%writefile 04-multi-block-loop.cu
#include <stdio.h>

__global__ void loop() {
  /* This idiomatic expression gives each thread
   * a unique index within the entire grid.
   */
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  printf("%d\n", i);
}

int main() {
  /* Additional execution configurations that would
   * work and meet the exercises contraints are:
   * <<<5, 2>>>
   * <<<10, 1>>> */
  loop<<<1, 10>>>();
  cudaDeviceSynchronize();
}

Writing 04-multi-block-loop.cu


In [12]:
!nvcc -arch=sm_70 -o multi-block-loop 04-multi-block-loop.cu -run

0
1
2
3
4
5
6
7
8
9


In [13]:
# trecho para estudo de gerenciamento de alocação de memória
# para alocar memória que possa ser acessada pelo host e pelo device, é necessário
# utilizar as funções CUDA de alocação e liberação de memória
%%writefile 05-double-elements.cu
#include <stdio.h>

void init(int *a, int N) {
  int i;
  for (i = 0; i < N; ++i) 
    a[i] = i;
}

__global__ void doubleElements(int *a, int N) {
  int i;
  i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < N)
    a[i] *= 2;
}

bool checkElementsAreDoubled(int *a, int N) {
  int i;
  for (i = 0; i < N; ++i)
    if (a[i] != i*2) 
      return false;
  return true;
}

int main() {
  int N = 1500;
  int *a;

  size_t size = N * sizeof(int);
  /* Use `cudaMallocManaged` to allocate pointer `a` available
   * on both the host and the device. */

  cudaMallocManaged(&a, size);
  init(a, N);

  size_t threads_per_block = 256;
  size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;

  doubleElements<<<number_of_blocks, threads_per_block>>>(a, N);
  cudaDeviceSynchronize();

  bool areDoubled = checkElementsAreDoubled(a, N);
  printf("All elements were doubled? %s\n", areDoubled ? "TRUE" : "FALSE");

  /* Use `cudaFree` to free memory allocated with `cudaMallocManaged`. */
  cudaFree(a);
}


Writing 05-double-elements.cu


In [14]:
!nvcc -arch=sm_70 -o double-elements 05-double-elements.cu -run

All elements were doubled? TRUE


In [15]:
# esta tarefa é sobre tratamento de incompatibilidade de configuração de bloco com o número de threads
# quando não conseguimos definir o número de threads multiplos do numero de blocos, 
# utilizamos uma expressão para gerar um id para a thread, criamos mais threads no total que o necessário
# e então verificamos pelo id quais threads serão utilizadas
%%writefile 06-mismatched-config-loop.cu
#include <stdio.h>

__global__ void initializeElementsTo(int initialValue, int *a, int N) {
  int i = threadIdx.x + blockIdx.x * blockDim.x;
  if (i < N) 
    a[i] = initialValue;
}

int main() {
  /* Do not modify `N`.  */
  int N = 1000;

  int *a;
  size_t size = N * sizeof(int);

  cudaMallocManaged(&a, size);

  /* Assume we have reason to want the number of threads
   * fixed at `256`: do not modify `threads_per_block`. */
  size_t threads_per_block = 256;

  /* The following is idiomatic CUDA to make sure there are at
   * least as many threads in the grid as there are `N` elements. */
  size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;

  int initialValue = 6;

  initializeElementsTo<<<number_of_blocks, threads_per_block>>>(initialValue, a, N);
  cudaDeviceSynchronize();

  /* Check to make sure all values in `a`, were initialized. */
  for (int i = 0; i < N; ++i) 
    if(a[i] != initialValue) {
      printf("FAILURE: target value: %d\t a[%d]: %d\n", initialValue, i, a[i]);
      cudaFree(a);
      exit(1);
    }
  printf("SUCCESS!\n");
  cudaFree(a);
}


Writing 06-mismatched-config-loop.cu


In [16]:
!nvcc -arch=sm_70 -o mismatched-config-loop 06-mismatched-config-loop.cu -run

SUCCESS!


In [17]:
# utilização do método de grid-stride loop
# Por exemplo, para uma matriz de 500 elementos e uma grade de 250 threads, a thread com índice 20 na grade seria:
# Realize sua operação no elemento 20 do array de 500 elementos
# Incrementar seu índice em 250, o tamanho da grade, resultando em 270
# Realize sua operação no elemento 270 do array de 500 elementos
# Incrementar seu índice em 250, o tamanho da grade, resultando em 520
# Como 520 agora está fora do alcance do array, o encadeamento interromperá seu trabalho
# com gridDim.x é possível verificar o numero de blocos em uma grid

%%writefile 07-grid-stride-double.cu
#include <stdio.h>

void init(int *a, int N) {
  int i;
  for (i = 0; i < N; ++i) 
    a[i] = i;
}

__global__ void doubleElements(int *a, int N) {
  /* Use a grid-stride loop so each thread does work
   * on more than one element in the array.  */
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = gridDim.x * blockDim.x;

  // printf("Numero de grids: %i\n", gridDim.x); 

  for (int i = idx; i < N; i += stride) 
    a[i] *= 2;
}

bool checkElementsAreDoubled(int *a, int N) {
  int i;
  for (i = 0; i < N; ++i) 
    if (a[i] != i*2) 
      return false;
  return true;
}

int main() {
  int N = 10000;
  int *a;

  size_t size = N * sizeof(int);
  cudaMallocManaged(&a, size);

  init(a, N);

  size_t threads_per_block = 256;
  size_t number_of_blocks = 32;

  doubleElements<<<number_of_blocks, threads_per_block>>>(a, N);
  cudaDeviceSynchronize();

  bool areDoubled = checkElementsAreDoubled(a, N);
  printf("All elements were doubled? %s\n", areDoubled ? "TRUE" : "FALSE");

  cudaFree(a);
}

Writing 07-grid-stride-double.cu


In [18]:
!nvcc -arch=sm_70 -o grid-stride-double 07-grid-stride-double.cu -run

All elements were doubled? TRUE


In [19]:
# tratamento de erros
# podemos utilizar cudaError_t para capturar um valor de erro de uma função
# ou usar cudaGetLastError para erros de inicialização do kernel
# ou cudaDeviceSynchronize para capturar erros em kernels assincronos

%%writefile 08-add-error-handling.cu
#include <stdio.h>

void init(int *a, int N) {
  int i;
  for (i = 0; i < N; ++i)
    a[i] = i;
}

__global__ void doubleElements(int *a, int N) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = gridDim.x * blockDim.x;

  /* The previous code (now commented out) attempted
   * to access an element outside the range of `a`.  */

  // for (int i = idx; i < N + stride; i += stride)
  for (int i = idx; i < N; i += stride)
    a[i] *= 2;
}

bool checkElementsAreDoubled(int *a, int N) {
  int i;
  for (i = 0; i < N; ++i)
    if (a[i] != i*2) 
      return false;
  return true;
}

int main() {
  int N = 10000;
  int *a;

  size_t size = N * sizeof(int);
  cudaMallocManaged(&a, size);

  init(a, N);

  /* The previous code (now commented out) attempted to launch
   * the kernel with more than the maximum number of threads per
   * block, which is 1024. */
  size_t threads_per_block = 1024;
  /* size_t threads_per_block = 2048; */
  size_t number_of_blocks = 32;

  cudaError_t syncErr, asyncErr;

  doubleElements<<<number_of_blocks, threads_per_block>>>(a, N);

  /* Catch errors for both the kernel launch above and any
   * errors that occur during the asynchronous `doubleElements`
   * kernel execution. */

  syncErr = cudaGetLastError();
  asyncErr = cudaDeviceSynchronize();

  /* Print errors should they exist. */

  if (syncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(syncErr));
  if (asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  bool areDoubled = checkElementsAreDoubled(a, N);
  printf("All elements were doubled? %s\n", areDoubled ? "TRUE" : "FALSE");

  cudaFree(a);
}

Writing 08-add-error-handling.cu


In [20]:
!nvcc -arch=sm_70 -o add-error-handling 08-add-error-handling.cu -run

All elements were doubled? TRUE


### Atividade: paralelizar o código disponibilizado

In [31]:
%%writefile  09-vector-add.cu
#include <stdio.h>

__global__ void initWith(float num, float *a, int N)
{
  int i;
  i = threadIdx.x;
  if (i < N)
  // for(int i = 0; i < N; ++i)
   {
    a[i] = num;
   }
}

__global__ void addVectorsInto(float *result, float *a, float *b, int N)
{
  int i;
  i = threadIdx.x;
  if (i < N){
    result[i] = a[i] + b[i];
  }
}

void checkElementsAre(float target, float *array, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(array[i] != target)
    {
      printf("FAIL: array[%d] - %0.0f does not equal %0.0f\n", i, array[i], target);
      exit(1);
    }
  }
  printf("SUCCESS! All values added correctly.\n");
}

int main()
{
  //const int N = 2<<20;
  // diminui a quantidade para conseguir executar
  const int N = 256;
  size_t size = N * sizeof(float);
  cudaError_t err;

  float *a;
  float *b;
  float *c;

  size_t threads_per_block = N;
  size_t number_of_blocks = 1;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith<<<number_of_blocks, threads_per_block>>>(3, a, N);
  initWith<<<number_of_blocks, threads_per_block>>>(4, b, N);
  initWith<<<number_of_blocks, threads_per_block>>>(0, c, N);
  err = cudaDeviceSynchronize();
  if (err != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(err));

  addVectorsInto<<<number_of_blocks, threads_per_block>>>(c, a, b, N);
  err = cudaDeviceSynchronize();
  if (err != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(err));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}

Overwriting 09-vector-add.cu


In [32]:
!nvcc -arch=sm_70 -o vector-add 09-vector-add.cu -run

SUCCESS! All values added correctly.
